به نام خدا

جست‌و‌جو

پروژه‌ی اول هوش مصنوعی

مهرماه ۱۳۹۹

علیرضا توکلی ۸۱۰۱۹۷۶۸۶

# هدف پروژه

هدف از انجام این پروژه مقایسه‌ی عمل‌کرد الگوریتم‌های گفته شده در سر کلاس؛ به طور مشخص‌تر

BFS, IDS, A*, Weighted A*

است. هم‌چنین می‌توان از این پروژه مفاهیم حالت‌ها و یال‌های بین آن‌ها را نیز تمرین کرد.

# توضیح کلی پروژه
در راستای هدف گفته شده، بازی‌ای وجود دارد که در آن ماری در تلاش است که غذاهای روی نقشه را بخورد. ما برای این کار می‌خواهیم مار کم‌ترین تعداد جابه‌جایی ممکن را داشته باشد. پس باید از روش‌های جست‌و‌جو در فضای حالت‌ها استفاده کنیم. 

# دیتاهای لود شده
دیتاهای لود شده سه ورودی داده شده هستند که برای بررسی عمل‌کرد الگوریتم‌ها مورد استفاده قرار می‌گیرند. هر کدام از این دیتاها میزان سختی حل مخصوص به خود را دارند تا الگوریتم‌ها در شرایط متفاوت بررسی شوند.

# نحوه‌ی مدل کردن مسئله
مسئله را به این صورت مدل خواهیم کرد که هر حالتی از مار در صفحه و هر حالتی از دانه‌ها را یک حالت می‌گیریم. هم‌چنین بین دو حالت یال می‌گذاریم اگر و تنها اگر به ازای یک  حرکت مار، از حالت اولیه به حالت ثانویه بتوان رسید.

حالت اولیه ما حالت ورودی است که از فایل می‌خوانیم و حالت نهایی ما هر حالتی است که در نقشه دانه وجود نداشته باشد؛ یا معادلا همه‌ی دانه‌ها خورده شده باشند.

In [95]:
import copy
import time
from collections import deque

In [142]:
class State:
    def __init__(self, snake, seeds):
        self.snake = snake
        self.seeds = seeds

    def __str__(self):
        return str([self.snake] + [self.seeds])

    def __repr__(self):
        return str([self.snake] + [self.seeds])

    def __hash__(self):
        return hash(str(self.snake + self.seeds))
    
    def __lt__(self, other):
        if self.snake[0][0] < other.snake[0][0]:
            return True
        elif self.snake[0][0] > other.snake[0][0]:
            return False
        else:
            return (self.snake[0][1] < other.snake[0][1])

In [97]:
dx = [-1, 0, +1, 0]
dy = [0, -1, 0, +1]
direction = "ULDR"

def expand(state, path, n, m): 
    x, y = state.snake[0]
    expandList = []
    
    for i in range(4):
        nx = (x + dx[i]) % n
        ny = (y + dy[i]) % m
        
        if len(state.snake) > 1 and [nx, ny] == state.snake[1]:
            continue
        
        crash = False
        for j in range(1, len(state.snake) - 1):
            if [nx, ny] == state.snake[j]:
                crash = True
                break
        
        if crash == True:
            continue
        
        indexSeed = -1
        for j in range(len(state.seeds)):
            if nx == state.seeds[j][0] and ny == state.seeds[j][1]:
                indexSeed = j
                break
                
        if indexSeed == -1:
            newSnake = [[nx, ny]] + state.snake[:-1]
            newSeeds = state.seeds
        else:
            newSnake = [[nx, ny]] + state.snake
            newSeeds = copy.deepcopy(state.seeds)
            if state.seeds[indexSeed][2] == 1:
                newSeeds.pop(indexSeed)
            else:
                newSeeds[indexSeed][2] -= 1
        newPath = path + direction[i]
        
        expandList.append([State(newSnake, newSeeds), newPath])
    return expandList

# BFS

## توضیح الگوریتم
صفی در اختیار داریم که پس از اکپسند کردن هر حالت، حالت‌های اکسپند شده که قبلا به آن‌ها نرسیده بودیم را به ته این صف اضافه می‌کند. برای این کار و بررسی سریع حالات همان‌طور که گفته شد، از تابع هش استفاده می‌شود که هر حالت را به یک عدد می‌برد. این طور می‌توان تمایز دو حالت را بررسی کرد. 

## تفاوت‌های این الگوریتم با الگوریتم‌های دیگر
از حافظه‌ی زیادی استفاده می‌کند. اما مطمئن خواهیم بود جواب بهینه است.

In [220]:
def bfs(firstState, n, m):
    queue = deque()
    queue.append([firstState, ''])
    expandSet = set()
    expandSet.add(hash(firstState))
    mojazaCount = 1
    count = 1
    while len(queue):
        state = queue.popleft()
        expandList = expand(state[0], state[1], n, m)
        for ex in expandList:
            count += len(expandList)
            if hash(ex[0]) not in expandSet:
                if len(ex[0].seeds) == 0:
                    return [ex[1], mojazaCount, count]
                queue.append(ex)
                mojazaCount += 1
                expandSet.add(hash(ex[0]))

In [221]:
t = 0
for _ in range(2):
    f = open("test1.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = bfs(State([[x, y]], seeds), n, m)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

LDLUUULLUULL 12 4735 29678
Time taken: 0.05362510681152344 seconds


In [223]:
t = 0
for _ in range(2):
    f = open("test2.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = bfs(State([[x, y]], seeds), n, m)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

URDLLUUUULULLLL 15 44184 288867
Time taken: 0.5359451770782471 seconds


In [224]:
t = 0
for _ in range(2):
    f = open("test3.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = bfs(State([[x, y]], seeds), n, m)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

URDDDRDRRDDRRRURRDLLLUULL 25 199157 1311521
Time taken: 2.5454245805740356 seconds


# DFS

## توضیح الگوریتم
در این بخش به جای اضافه کردن حالت‌های اکسپند شده به ته صف، آن‌ها را به سر صف اضافه می‌کنیم با این تفاوت که حواسمان هست که از ال مرحله جلوتر نرویم. حال اگر ال را از ابتدا شروع کرده و تا وقتی که به جواب نرسیدیم زیاد کنیم، بدیهتا همان‌طور که سر کلاس گفته شد، به جواب می‌رسیم.

برای بررسی این که حالت را قبلا دیده‌ایم یا نه و اگر دیده‌ایم، با چه فاصله‌ای دیده‌ایم از دیکشنری که این عملیات‌ها را از اردر لاگ ان انجام می‌دهد استفاده می‌کنیم.

## تفاوت این الگوریتم با الگوریتم‌های دیگر
این الگوریتم زمان بیش‌تری می‌برد زیرا باید به ازای ارتفاع‌های مختلف بعضی از حالات را چندین بار پیمایش کند. اما این زمان به صورتی نیست که اردر زمانی را بد کند و همان چیزی که برای بی‌اف‌اس بود می‌ماند.
بر خلاف بقیه نیازی به ذخیره سازی تمامی حالات ندارد؛ پس می‌توان گفت که حافظه‌ی کم‌تری می‌برد.

In [260]:
def dfs(firstState, n, m, l):
    queue = deque()
    queue.append([firstState, ''])
    dis = {hash(firstState):0}
    mojazaCount = 1
    count = 1
    while len(queue):
        state = queue.pop()
        if len(state[1]) >= l:
            continue
        expandList = expand(state[0], state[1], n, m)
        for ex in expandList:
            count += len(expandList)
            if (hash(ex[0]) not in dis) or dis[hash(ex[0])] > len(ex[1]):
                if len(ex[0].seeds) == 0:
                    return [ex[1], mojazaCount, count]
                mojazaCount += 1
                queue.append(ex)
                dis[hash(ex[0])] = len(ex[1])
    return ['', mojazaCount, count]

In [279]:
def IDS(state, n, m):
    i = 0
    mojazaCount = 0
    count = 0
    while True:
        ans = dfs(state, n, m, i)
        mojazaCount += ans[1]
        count += ans[2]
        if ans[0] != '':
            return ans
        i += 1

In [280]:
t = 0
for _ in range(2):
    f = open("test1.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = IDS(State([[x, y]], seeds), n, m)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

DLDRRDRRDRDD 12 8192 44137
Time taken: 0.26406431198120117 seconds


In [281]:
t = 0
for _ in range(2):
    f = open("test2.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = IDS(State([[x, y]], seeds), n, m)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

RULDLULUUULLLLU 15 32292 181719
Time taken: 2.759732484817505 seconds


In [282]:
t = 0
for _ in range(2):
    f = open("test3.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = IDS(State([[x, y]], seeds), n, m)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

RURDDDRRDRRRDDRRULLDLLLUU 25 398609 2554085
Time taken: 23.737545490264893 seconds


# A*

## توضیح الگوریتم
در این الگوریتم از صفی با خاصیت مرتب بودن استفاده می‌کنیم. در قطعه کد پایین این صف را با هیپ پیاده‌سازی کرده‌ایم. هر حالت عدد که در این صف دارد برابر با فاصله‌اش تا ابتدای مسیر به علاوه‌ی تابع اچی که ما آن‌ها تخمین می‌زنیم. حال ادامه‌ی کار دقیقا مانند بی‌اف‌اس است.

## تفاوت این الگوریتم با الگوریتم‌های دیگر
به دلیل داشتن تابع پیش‌بینی، به سرعت آن خیلی کمک شده است ولی برای این که این کمک انجام شود، حتما باید اثبات‌هایی انجام شود. در حالت اول به دلیل کانسیستنسی، نیازی به دیدن حالت‌های تکراری نیست و این کار باعث بهتر شدن آن نسبت به بی‌اف‌اس می‌شود اما در حالت غیر کانسیستنی، باید حالت‌های تکراری را نیز بررسی کنیم که نیازمند دیکشنری است.

In [125]:
import heapq

In [233]:
def aStar(firstState, n, m, h):
    queue = []
    heapq.heappush(queue, [h(firstState, n, m), firstState, ''])
    expandSet = set()
    expandSet.add(hash(firstState))
    mojazaCount = 1
    count = 1
    while queue:
        distance, state, path = heapq.heappop(queue)
        expandList = expand(state, path, n, m)
        for ex in expandList:
            count += len(expandList)
            if hash(ex[0]) not in expandSet:
                if len(ex[0].seeds) == 0:
                    return [ex[1], mojazaCount, count]
                mojazaCount += 1
                heapq.heappush(queue, [h(ex[0], n, m) + len(ex[1]), ex[0], ex[1]])
                expandSet.add(hash(ex[0]))

## تعداد دانه‌های خورده نشده

تابع مورد نظرمان را تعداد دانه‌های خورده نشده تعریف می‌کنیم. می‌دانیم که برای خوردن هر دانه حداقل یک عملیات نیاز است، پس این تابع ما برای هر حالت، کم‌تر مساوی کوچک‌ترین مسیر این حالت تا یکی از حالات نهایی است.
پس admissable است.

حال ادعا داریم این الگوریتم consistency هم دارد.

زیرا هر دو حالت را بگیرید، فرض کنید از اولی

A

به دومی برویم. از آن‌جایی که به حالت بعدی رفتیم، پس تعداد دانه‌های روی زمین کم‌تر مساوی شده است.حال فاصله حالت اول تا یکی از حالات نهایی منهای فاصله‌ی حالت دوم تا یکی از حالات نهایی، حتما کم‌تر مساوی فاصله‌ی بین این دو حالت است. زیرا اگر نباشد، ابتدا از حالت اول به حالت دوم می‌رویم و سپس تمام دانه‌ها را می‌خوریم و مسیر کوتاه‌تری خواهیم داشت.

In [193]:
def heuristic1(state, n, m):
    return len(state.seeds)

In [234]:
t = 0
for _ in range(2):
    f = open("test1.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = aStar(State([[x, y]], seeds), n, m, heuristic1)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

LDRDRDRDRDRD 12 2825 15482
Time taken: 0.038837432861328125 seconds


In [235]:
t = 0
for _ in range(2):
    f = open("test2.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = aStar(State([[x, y]], seeds), n, m, heuristic1)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

URDLLUUULULLLUL 15 26669 156380
Time taken: 0.4536869525909424 seconds


In [236]:
t = 0
for _ in range(2):
    f = open("test3.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = aStar(State([[x, y]], seeds), n, m, heuristic1)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

URDDDRRDRDRRRDRRULLDLLUUL 25 128170 806766
Time taken: 2.461205244064331 seconds


## فاصله تا اولین دانه

مار باید در هر حالت دانه‌ای که سر لیست دانه‌هاست را بخورد. کوتاه‌ترین راه راه منهتنی‌ایست که ممکن است از دیوارها نیز رد بشود. اگر تعدادی دانه‌ی دیگر در سر راه بخورد که مسیرش طولانی‌تر نیز می‌شود. پس اثبات کردیم تابع ما ادمیزبل است.

حال از آن‌جایی که این تابع کانسیستند نیست، برای پیدا کردن جواب بهینه، حتما می‌بایست حالت‌های تکراری را نیز مشاهده کرد.

In [237]:
def heuristic2(state, n, m):
    x, y = state.snake[0]
    a = state.seeds[0][0]
    b = state.seeds[0][1]
    return min((x - a) % n, (a - x) % n) + min((y - b) % m, (b - y) % m)

In [238]:
t = 0
for _ in range(2):
    f = open("test1.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = aStar(State([[x, y]], seeds), n, m, heuristic2)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

DLULUULLUULL 12 2955 15642
Time taken: 0.07639646530151367 seconds


In [239]:
t = 0
for _ in range(2):
    f = open("test2.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = aStar(State([[x, y]], seeds), n, m, heuristic2)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

RULLDLUUUUULLLL 15 15577 93609
Time taken: 0.24728405475616455 seconds


In [240]:
t = 0
for _ in range(2):
    f = open("test3.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = aStar(State([[x, y]], seeds), n, m, heuristic2)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

URDDRDRRDDDRRRRRULLDLUULL 25 79028 498013
Time taken: 1.5778253078460693 seconds


# Weighted A*

## توضیح الگوریتم
در این الگوریتم ما به جای استفاده از تابع اچ از ضریبی از آن استفاده می‌کنیم.

## تفاوت این الگوریتم با الگوریتم‌های دیگر
در این الگوریتم سرعت بسیار زیاد‌تر شده است. زیرا تابع پیش‌بینی ما ارزش بالاتری پیدا کرده است. اما دیگر اثبات‌هایی که برای توابعمان کردیم کاربرد ندارد. پس ممکن است جواب بهینه را نداشته باشیم.

In [245]:
def weightedAStar(firstState, n, m, h, alpha):
    queue = []
    heapq.heappush(queue, [alpha * h(firstState, n, m), firstState, ''])
    expandSet = set()
    expandSet.add(hash(firstState))
    mojazaCount = 1
    count = 1
    while queue:
        distance, state, path = heapq.heappop(queue)
        expandList = expand(state, path, n, m)
        for ex in expandList:
            count += len(expandList)
            if hash(ex[0]) not in expandSet:
                if len(ex[0].seeds) == 0:
                    return [ex[1], mojazaCount, count]
                mojazaCount += 1
                heapq.heappush(queue, [alpha * h(ex[0], n, m) + len(ex[1]), ex[0], ex[1]])
                expandSet.add(hash(ex[0]))

## alpha = 2

In [246]:
alpha = 2

### heuristic1

In [247]:
t = 0
for _ in range(2):
    f = open("test1.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = weightedAStar(State([[x, y]], seeds), n, m, heuristic1, alpha)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

LDLUUULLLUUL 12 1513 6985
Time taken: 0.02003800868988037 seconds


In [248]:
t = 0
for _ in range(2):
    f = open("test2.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = weightedAStar(State([[x, y]], seeds), n, m, heuristic1, alpha)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

RULDLUUULUULLLL 15 11575 57208
Time taken: 0.17649626731872559 seconds


In [249]:
t = 0
for _ in range(2):
    f = open("test3.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = weightedAStar(State([[x, y]], seeds), n, m, heuristic1, alpha)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

URDDDRRDRDDRRRURRDLLLULUL 25 73859 441701
Time taken: 1.3258882761001587 seconds


### heuristic 2

In [250]:
t = 0
for _ in range(2):
    f = open("test1.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = weightedAStar(State([[x, y]], seeds), n, m, heuristic2, alpha)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

DLULDRRRDDRRD 13 3075 16864
Time taken: 0.04879724979400635 seconds


In [251]:
t = 0
for _ in range(2):
    f = open("test2.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = weightedAStar(State([[x, y]], seeds), n, m, heuristic2, alpha)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

URDLLUUUULULLLL 15 33144 221242
Time taken: 0.5930299758911133 seconds


In [252]:
t = 0
for _ in range(2):
    f = open("test3.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = weightedAStar(State([[x, y]], seeds), n, m, heuristic2, alpha)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

RURDDDRRDRULDDRDRRRRULLLDR 26 43655 262909
Time taken: 0.7639806270599365 seconds


همان‌طور که گفته شد. به ازای این آلفا، بهینه بودن جواب مقدار کمی خراب‌شده است. اما سرعت آن زیاد‌تر شده است.

## alpha = 5

In [253]:
alpha = 5

### heuristic1

In [254]:
t = 0
for _ in range(2):
    f = open("test1.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = weightedAStar(State([[x, y]], seeds), n, m, heuristic1, alpha)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

LLUULLUULLDDDDD 15 1410 10013
Time taken: 0.02353978157043457 seconds


In [255]:
t = 0
for _ in range(2):
    f = open("test2.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = weightedAStar(State([[x, y]], seeds), n, m, heuristic1, alpha)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

LURDRUULULULLLLUL 17 1863 7223
Time taken: 0.026841282844543457 seconds


In [256]:
t = 0
for _ in range(2):
    f = open("test3.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = weightedAStar(State([[x, y]], seeds), n, m, heuristic1, alpha)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

RURDDDRRDLURDRRDDRRURULLDD 26 18504 97368
Time taken: 0.2730644941329956 seconds


### heuristic 2

In [257]:
t = 0
for _ in range(2):
    f = open("test1.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = weightedAStar(State([[x, y]], seeds), n, m, heuristic2, alpha)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

RUURRDRDDLLLLL 14 2114 10683
Time taken: 0.03212594985961914 seconds


In [258]:
t = 0
for _ in range(2):
    f = open("test2.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = weightedAStar(State([[x, y]], seeds), n, m, heuristic2, alpha)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

RULDLUUUULULLLL 15 65446 502370
Time taken: 1.1675982475280762 seconds


In [259]:
t = 0
for _ in range(2):
    f = open("test3.txt", "r")
    n, m = map(int, f.readline().strip().split(','))
    x, y = map(int, f.readline().strip().split(','))
    q = int(f.readline().strip().split(',')[0])
    seeds = []
    for i in range(q):
        seeds.append(list(map(int, f.readline().strip().split(','))))
    
    tic = time.time()
    ans = weightedAStar(State([[x, y]], seeds), n, m, heuristic2, alpha)
    toc = time.time()
    t += toc - tic
print(ans[0], len(ans[0]), ans[1], ans[2])
print("Time taken:", t / 2, "seconds")

RURDRRDDDLULDRRRRDDRURRDLDLU 28 14885 90361
Time taken: 0.23508822917938232 seconds


# Test1.txt

| Algorithm/Feature | Distance | Path | Seen States | Mojaza Seen States | Time
| --- | --- | --- | --- | --- | --- |
| BFS | 12 | LDLUUULLUULL | 29678 | 4735 | 0.0532534122467041
| DFS | 12 | DLDRRDRRDRDD | 44137 | 8192 | 0.26406431198120117
| A*1 | 12 | LDRDRDRDRDRD | 15482 | 2825 | 0.038837432861328125
| A*2 | 12 | DLULUULLUULL | 15642 | 2955 | 0.07639646530151367
| Weighted2 | 13 | DLULDRRRDDRRD | 16864 | 3075 | 0.04879724979400635
| Weighted5 | 14 | RUURRDRDDLLLLL | 10683 | 2114 | 0.03212594985961914

# Test2.txt

| Algorithm/Feature | Distance | Path | Seen States | Mojaza Seen States | Time
| --- | --- | --- | --- | --- | --- |
| BFS | 15 | URDLLUUUULULLLL | 288867 | 44184 | 0.5359451770782471
| DFS | 15 | RULDLULUUULLLLU | 181719 | 32292 | 2.759732484817505
| A*1 | 15 | URDLLUUULULLLUL | 156380 | 26669 | 0.4536869525909424
| A*2 | 15 | RULLDLUUUUULLLL | 93609 | 15577 | 0.24728405475616455
| Weighted2 | 15 | URDLLUUUULULLLL | 221242 | 33144 | 0.5930299758911133
| Weighted5 | 15 | RULDLUUUULULLLL | 502370 | 65446 | 1.1675982475280762

# Test3.txt

| Algorithm/Feature | Distance | Path | Seen States | Mojaza Seen States | Time
| --- | --- | --- | --- | --- | --- |
| BFS | 25 | URDDDRDRRDDRRRURRDLLLUULL | 1311521 | 199157 | 2.5454245805740356
| DFS | 25 | RURDDDRRDRRRDDRRULLDLLLUU | 2554085 | 398609 | 23.737545490264893
| A*1 | 25 | URDDDRRDRDRRRDRRULLDLLUUL | 806766 | 128170 | 2.461205244064331
| A*2 | 25 | URDDRDRRDDDRRRRRULLDLUULL | 498013 | 79028 | 1.5778253078460693
| Weighted2 | 26 | RURDDDRRDRULDDRDRRRRULLLDR | 262909 | 43655 | 0.7639806270599365
| Weighted5 | 28 | RURDRRDDDLULDRRRRDDRURRDLDLU | 90361 | 14885 | 0.23508822917938232

# نتیجه‌گیری
همان‌طور که مشاهده می‌شود، حرف‌هایی که قبل از هر الگوریتم و تفاوت‌های بین الگوریتم‌های گفته شده درست می‌باشد.